In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np 
import tqdm
from arcs.setup_functions import ReactionGibbsandEquilibrium
from monty.serialization import loadfn
from arcs.setup_functions import GraphGenerator
from arcs.setup_functions import GetEnergyandVibrationsVASP

In [3]:
dft_data = loadfn('../tests/test_dft_data.json')
rge = ReactionGibbsandEquilibrium(dft_data)

applied_reactions = [] 
for reaction in tqdm.tqdm(dft_data['reactions'].values()):
    g_k_dict = rge.get_reaction_gibbs_and_equilibrium(reaction=reaction,temperature=100,pressure=1)
    g_k_dict['r'] = reaction
    applied_reactions.append(g_k_dict)


gg = GraphGenerator(applied_reactions = applied_reactions)
graph = gg.generate_multidigraph(temperature=100)

  0%|          | 0/501 [00:00<?, ?it/s]/Users/badw/github-projects/arcs/arcs/setup_functions.py:255: RuntimeWarning: overflow encountered in exp
  K = np.exp(
100%|██████████| 501/501 [00:00<00:00, 3184.77it/s]


In [12]:
from arcs.setup_functions import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph)
gic.specific_random(compounds=['CO2'])

{'H2S': 0,
 'O2': 0,
 'H2SO3': 0,
 'H2O': 0,
 'H2': 0,
 'CO2': 9.209381237749511e-07,
 'CH4': 0,
 'CH3COOH': 0,
 'CO': 0,
 'CH3OH': 0,
 'H2CO': 0,
 'HNO2': 0,
 'HNO3': 0,
 'CH2O2': 0,
 'S8': 0,
 'SO2': 0,
 'H2SO4': 0,
 'CH3CHO': 0,
 'CH3CH2OH': 0,
 'N2': 0,
 'NO': 0,
 'NO2': 0,
 'NOHSO4': 0,
 'NH3': 0}

In [13]:
gic.update_ic({'H2O':1e-6})

{'H2S': 0,
 'O2': 0,
 'H2SO3': 0,
 'H2O': 1e-06,
 'H2': 0,
 'CO2': 9.209381237749511e-07,
 'CH4': 0,
 'CH3COOH': 0,
 'CO': 0,
 'CH3OH': 0,
 'H2CO': 0,
 'HNO2': 0,
 'HNO3': 0,
 'CH2O2': 0,
 'S8': 0,
 'SO2': 0,
 'H2SO4': 0,
 'CH3CHO': 0,
 'CH3CH2OH': 0,
 'N2': 0,
 'NO': 0,
 'NO2': 0,
 'NOHSO4': 0,
 'NH3': 0}